In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Clone MMDetection from the main branch (or specify a valid tag if needed)
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -v -e .

Cloning into 'mmdetection'...
remote: Enumerating objects: 38023, done.
remote: Total 38023 (delta 0), reused 0 (delta 0), pack-reused 38023 (from 1)
Receiving objects: 100% (38023/38023), 63.25 MiB | 36.13 MiB/s, done.
Resolving deltas: 100% (26223/26223), done.
/content/mmdetection
Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Obtaining file:///content/mmdetection
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-frjth1su/mmdet.egg-info
  writing /tmp/pip-pip-egg-info-frjth1su/mmdet.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-frjth1su/mmdet.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-frjth1su/mmdet.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-frjth1su/mmdet.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-frjth1su/mmdet.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-frjth1su/mm

In [ ]:
!pip install -U openmim
!mim install mmengine
!mim install "mmcv==2.1.0"

Looking in links: https://download.openmmlab.com/mmcv/dist/cu124/torch2.6.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu124/torch2.6.0/index.html
  Using cached mmcv-2.1.0.tar.gz (471 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for mmcv: filename=mmcv-2.1.0-cp311-cp311-linux_x86_64.whl size=38291390 sha256=d1dfb0cc2bb04741efc018d8831d68d8d7de02451b831172c167de1f7adfb285
  Stored in directory: /root/.cache/pip/wheels/57/22/96/9bcbfeaa838c1b705a4b2ac082539767fbcc79fa07ec01abcd
Successfully built mmcv


In [ ]:
!mkdir -p configs/faster_rcnn
!wget https://raw.githubusercontent.com/open-mmlab/mmdetection/main/configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py -O configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py

--2025-04-15 19:33:12--  https://raw.githubusercontent.com/open-mmlab/mmdetection/main/configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177 [text/plain]
Saving to: ‘configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py’

configs/faster_rcnn 100%[===================>]     177  --.-KB/s    in 0s      

2025-04-15 19:33:12 (13.8 MB/s) - ‘configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py’ saved [177/177]



In [ ]:
!git clone -b 3.3.0 https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -v -e .

fatal: destination path 'mmdetection' already exists and is not an empty directory.
/content/mmdetection
Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Obtaining file:///content/mmdetection
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-oct18gb_/mmdet.egg-info
  writing manifest file '/tmp/pip-pip-egg-info-oct18gb_/mmdet.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-oct18gb_/mmdet.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Attempting uninstall: mmdet
    Found existing installation: mmdet 3.3.0
    Uninstalling mmdet-3.3.0:
      Removing file or directory /usr/local/lib/python3.11/dist-packages/mmdet.egg-link
      Removing pth entries from /usr/local/lib/python3.11/dist-packages/easy-install.pth:
      Removing entry: /content/mmdetection
      Successfully uninstalled mmdet-3.3.0
  Running setup.py develop for mmdet
    Running command python setup.py develop

In [ ]:
from mmengine.config import Config
from mmengine.runner import Runner
import os

# === Paths (Google Drive Mounted) ===
drive_root = '/content/drive/MyDrive'
dataset_root = os.path.join(drive_root, '7_coco')

train_ann = os.path.join(dataset_root, 'train', 'train_annotations.json')
val_ann = os.path.join(dataset_root, 'train', 'val_annotations.json')
train_img = os.path.join(dataset_root, 'train', 'images')
val_img = train_img  # using same image folder

# === Load base config ===
cfg = Config.fromfile('configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py')

cfg.dataset_type = 'CocoDataset'
cfg.data_root = dataset_root
cfg.metainfo = {'classes': ('PET', 'PP', 'HDPE')}

# === Train dataset ===
cfg.train_dataloader.dataset = dict(
    type='CocoDataset',
    data_root=dataset_root,
    ann_file=train_ann,
    data_prefix=dict(img='train/images'),
    metainfo=cfg.metainfo,
    filter_cfg=dict(filter_empty_gt=True, min_size=32),
    pipeline=cfg.train_dataloader.dataset.pipeline
)

# === Validation dataset ===
cfg.val_dataloader.dataset = dict(
    type='CocoDataset',
    data_root=dataset_root,
    ann_file=val_ann,
    data_prefix=dict(img='train/images'),
    metainfo=cfg.metainfo,
    test_mode=True,
    pipeline=cfg.val_dataloader.dataset.pipeline
)

# === Also use val set for test_dataloader ===
cfg.test_dataloader = cfg.val_dataloader

# === Model ===
cfg.model.roi_head.bbox_head.num_classes = 3
cfg.device = 'cuda'

# === Output ===
cfg.work_dir = os.path.join(drive_root, 'faster_rcnn_output_2')
os.makedirs(cfg.work_dir, exist_ok=True)

# === Train parameters ===
cfg.train_cfg.max_epochs = 20
cfg.default_hooks.checkpoint.interval = 1
cfg.default_hooks.logger.interval = 10
cfg.randomness = dict(seed=42, deterministic=False)

# === Fix evaluator annotation file paths ===
cfg.val_evaluator.ann_file = val_ann
cfg.test_evaluator.ann_file = val_ann

# === Evaluate after every epoch ===
cfg.val_cfg = dict(interval=1)

# === Evaluators ===
cfg.val_evaluator = dict(
    type='CocoEvaluator',
    ann_file=val_ann,
    metric='bbox'
)
cfg.test_evaluator = cfg.val_evaluator  # use same evaluator

# === Learning rate scheduler ===
cfg.param_scheduler = [
    dict(
        type='MultiStepLR',
        milestones=[7, 9],
        gamma=0.1,
        by_epoch=True
    )
]

# === Early stopping hook ===
cfg.default_hooks.early_stop = dict(
    type='EarlyStoppingHook',
    monitor='coco/bbox_mAP',
    rule='greater',
    patience=3
)

# === Start training ===
runner = Runner.from_cfg(cfg)
runner.train()

04/15 19:50:28 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 42
    GPU 0: NVIDIA A100-SXM4-40GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.5, V12.5.82
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.6.0+cu124
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2024.2-Product Build 20240605 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.5.3 (Git Hash 66f0cb9eb66affd2da3bf5f8d897376f04aae6af)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.5
  - Built with CUDA Runtime 12.4
  - NVCC architecture 

TypeError: ValLoop.__init__() got an unexpected keyword argument 'interval'

In [ ]:
from mmengine.config import Config
from mmengine.runner import Runner
import os

# === Paths (Google Drive Mounted) ===
drive_root = '/content/drive/MyDrive'
dataset_root = os.path.join(drive_root, '7_coco')

train_ann = os.path.join(dataset_root, 'train', 'train_annotations.json')
val_ann = os.path.join(dataset_root, 'train', 'val_annotations.json')
train_img = os.path.join(dataset_root, 'train', 'images')
val_img = train_img  # using same image folder

# === Load base config ===
cfg = Config.fromfile('configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py')

cfg.dataset_type = 'CocoDataset'
cfg.data_root = dataset_root
cfg.metainfo = {'classes': ('PET', 'PP', 'HDPE')}

# === Train dataset ===
cfg.train_dataloader.dataset = dict(
    type='CocoDataset',
    data_root=dataset_root,
    ann_file=train_ann,
    data_prefix=dict(img='train/images'),
    metainfo=cfg.metainfo,
    filter_cfg=dict(filter_empty_gt=True, min_size=32),
    pipeline=cfg.train_dataloader.dataset.pipeline
)

# === Validation dataset ===
cfg.val_dataloader.dataset = dict(
    type='CocoDataset',
    data_root=dataset_root,
    ann_file=val_ann,
    data_prefix=dict(img='train/images'),
    metainfo=cfg.metainfo,
    test_mode=True,
    pipeline=cfg.val_dataloader.dataset.pipeline
)

# === Also use val set for test_dataloader ===
cfg.test_dataloader = cfg.val_dataloader

# === Model ===
cfg.model.roi_head.bbox_head.num_classes = 3
cfg.device = 'cuda'

# === Output ===
cfg.work_dir = os.path.join(drive_root, 'faster_rcnn_output_2')
os.makedirs(cfg.work_dir, exist_ok=True)

# === Train parameters ===
cfg.train_cfg.max_epochs = 10
cfg.default_hooks.checkpoint.interval = 1
cfg.default_hooks.logger.interval = 10
cfg.randomness = dict(seed=42, deterministic=False)

# === Fix evaluator annotation file paths ===
cfg.val_evaluator.ann_file = val_ann
cfg.test_evaluator.ann_file = val_ann

# === Start training ===
runner = Runner.from_cfg(cfg)
runner.train()

04/15 19:57:42 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 42
    GPU 0: NVIDIA A100-SXM4-40GB
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.5, V12.5.82
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.6.0+cu124
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2024.2-Product Build 20240605 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.5.3 (Git Hash 66f0cb9eb66affd2da3bf5f8d897376f04aae6af)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.5
  - Built with CUDA Runtime 12.4
  - NVCC architecture 

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


04/15 19:57:45 - mmengine - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

04/15 19:57:45 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
04/15 19:57:45 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
04/15 19:57:45 - mmengine - INFO - Checkpoints will be saved to /content/drive/MyDrive/faster_rcnn_output_2.
04/15 19:57:50 - mmengine - INFO - Epoch(train)  [1][ 10/920]  lr: 3.8036e-04  eta: 1:21:25  time: 0.5316  data_time: 0.1799  memory: 2847  loss: 1.4604  loss_rpn_cls: 0.6993  loss_rpn_bbox: 0.0365  loss_cls: 0.6991  acc: 97.0703  loss_bbox: 0.0254
04/15 19:57:54 - mmengine - INFO - Epoch(train)  [1][ 20/920]  lr: 7.8076e-04  eta: 1:08:04  time: 0.4450  data_time: 0.2148  memory: 2848  loss: 1.2349  loss_rpn_cls: 0.6770  loss_rpn_bb

FasterRCNN(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=Tru